# <font color='red'>Sistema de recomendação baseado em conteúdo</font>
## <font color='orangered'>Por Fran Mateus</font>
## <font color='orange'>Em 02-01-2025</font>

## <font color='darkblue'>Contexto</font>

Neste notebook, criaremos um **sistema de recomendação de filmes** baseado em conteúdo, ou seja, fundamentado na similaridade entre os itens. Para isso, serão usadas funções das bibliotecas *scikit-learn* e *natural language toolkit* (nltk). O código disponibilizado abaixo pode ser adaptado para outros tipos de recomendação como livros e artigos.

## <font color='darkblue'>Parte 1</font>

### I - Bibliotecas e funções necessárias

In [1]:
import pandas as pd
import numpy as np
import ast #Abstract Syntax Trees

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.stem.porter import PorterStemmer
pd.options.mode.chained_assignment = None

C:\Users\mfran\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\mfran\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\mfran\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### II - Dados dos filmes

In [2]:
# Leitura dos dados
data_movies = pd.read_csv("dados/data_movies.csv")
data_crew = pd.read_csv("dados/data_crew.csv")

# Shapes
print(data_movies.shape)
print(data_crew.shape)

(4803, 20)
(4803, 4)


In [3]:
# Dataset dos filmes
data_movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
# Dataset do elenco, direção e outros membros da equipe
data_crew.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [5]:
# Mesclando os datasets
data_movies = data_movies.merge(data_crew, on = 'title')
print(data_movies.shape)
data_movies.head(1)

(4809, 23)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [6]:
# Selecionando as variáveis mais relevantes; a maioria, com dados textuais.
data_movies = data_movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
print(data_movies.shape)
data_movies.head(1)

(4809, 7)


,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


## <font color='darkblue'>Parte 2</font>

## Limpeza e preparação dos dados textuais

In [7]:
# Removendo valores ausentes
data_movies.dropna(inplace = True)
data_movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [8]:
# Checando dados duplicados
data_movies.duplicated().sum()

0

## Processamento de texto

### III - Ajuste das variáveis Genres (generos) e Keywords (palavras-chave) dos filmes.

In [9]:
# Função de conversão usando ast.literal_eval
def converter(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [10]:
# Convertendo as variáveis Genres e Keywords
data_movies['genres'] = data_movies['genres'].apply(converter)
data_movies['keywords'] = data_movies['keywords'].apply(converter)

### IV - Ajuste da variável Cast (elenco)

In [11]:
# Função de conversão da variável Cast para obter, apenas, uma parte do elenco. Neste caso, 10 deles.
def converter10(obj):
    
    L = []  
    counter = 0  
    
    for i in ast.literal_eval(obj):  
        if counter != 10:  
            L.append(i['name'])  
            counter += 1  
        else:
            break  
            
    return L  

In [12]:
# Convertendo a variável Cast
data_movies['cast'] = data_movies['cast'].apply(converter10)

### V - Ajuste da variável Crew (equipe)

In [13]:
# Função de conversão para obter apenas o nome do diretor do filme
def fetch_director(obj):
    
    L = []
    
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    
    return L

In [14]:
# Convertendo a variável Crew
data_movies['crew'] = data_movies['crew'].apply(fetch_director)

### VI - Juntando os dados numa única coluna

In [15]:
# 1- Separa a string da variável Overview por espaço em branco
data_movies['overview'] = data_movies['overview'].apply(lambda x:x.split())

In [16]:
# 2- Remove os espaços de cada variável
data_movies['genres'] = data_movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
data_movies['keywords'] = data_movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
data_movies['cast'] = data_movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
data_movies['crew'] = data_movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [17]:
# 3- Cria uma coluna (um vetor de strings) juntando os dados das 5 variáveis acima
data_movies['tags'] = data_movies['overview'] + \
                        data_movies['genres'] + \
                        data_movies['keywords'] + \
                        data_movies['cast'] + \
                        data_movies['crew']

new_data_movies = data_movies[['movie_id', 'title', 'tags']]

In [18]:
# 4- Tratamento da nova variável
new_data_movies['tags'] = new_data_movies['tags'].apply(lambda x:" ".join(x))
new_data_movies['tags'] = new_data_movies['tags'].apply(lambda x:x.lower())

### VII - Parse e Vetorização

In [19]:
# Cria o parser
parser_ps = PorterStemmer()

In [20]:
# Função de stemming
def stem(text):
    
    y = []
    
    for i in text.split():
        y.append(parser_ps.stem(i))
    
    return " ".join(y)

In [21]:
# Aplica a função à coluna de tags
new_data_movies['tags'] = new_data_movies['tags'].apply(stem)

In [22]:
# Visualizando o dataset final, antes da sua vetorização
new_data_movies.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."


In [23]:
# Criando o vetorizador (matriz de contagem de dados) 
# Quantidade máxima das 6000 palavras mais frequentes
cv = CountVectorizer(max_features = 6000, stop_words = 'english')

In [24]:
# Cria os vetores para as tags
vectors = cv.fit_transform(new_data_movies['tags']).toarray()

In [25]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## <font color='darkblue'>Parte 3 - Sistema de Recomendação</font>

### VIII - Calcula a similaridade entre os vetores

In [26]:
# Uso da similaridade de cosseno para encontrar a distância entre os vetores
similaridades = cosine_similarity(vectors)

### IX - Desenvolve o sistema de recomendação

In [27]:
# Função para construção do sistema de recomendação
# 1- Obtém-se o índice do título passado como argumento; ou seja, o filme que o usuário assistiu.
# 2- Verifica os filmes com vetores de menor distância (maior similaridade) para o filme passado como argumento.
# 3- Seleciona os filmes com menor distância / maior similaridade para recomendar ao usuário.

def sistema_recomendacao(movie):
    
    index = new_data_movies[new_data_movies['title'] == movie].index[0]
    
    distances = sorted(list(enumerate(similaridades[index])), reverse = True, key = lambda x: x[1])
    
    for i in distances[1:4]:
        print(new_data_movies.iloc[i[0]].title)

### X - Testa o Sistema de Recomendação

#### <font color='orangered'>Top 3 filmes recomendados para quem assistiu "007 Contra Spectre"</font>

In [28]:
sistema_recomendacao('Spectre')

Skyfall
Quantum of Solace
Never Say Never Again


**Comentário:** 
- Os dois primeiros filmes foram protagonizados pelo 007 de Daniel Craig, a M de Judi Dench e são histórias que antecedem *007 Contra Spectre*. 
- O terceiro filme foi protagonizado pelo 007 de Sean Connery e menciona a organização criminosa SPECTRE.

#### <font color='orangered'>Top 3 filmes recomendados para quem assistiu "Wolverine - Imortal"</font>

In [29]:
sistema_recomendacao('The Wolverine')

X2
X-Men
Iron Man 2


**Comentário:** 
- Os dois primeiros filmes são protagonizados por Hugh Jackman como Wolverine/Logan e por outros personagens vistos em *Wolverine - Imortal*, como Jean (Famke Jamsen) e, no final, Professor Charles Xavier (Patrick Stewart).
- No caso de *Iron Man 2*, a principal semelhança aparente é o fato da história também ser da Marvel.

#### <font color='orangered'>Top 3 filmes recomendados para quem assistiu "Um Lugar Chamado Notting Hill"</font>

In [30]:
sistema_recomendacao('Notting Hill')

Boynton Beach Club
It's All Gone Pete Tong
Love Actually


**Comentário:** 
- Entre os 3 primeiros filmes, *Love Actually* (em português, *Simplesmente Amor*, 2003), apresenta mais semelhanças com *Notting Hill* do que os 2 primeiros do ranking acima: o mesmo diretor, comédia romântica, Londres como locação e Hugh Grant no elenco.
- Similar ao filme *Notting Hill*, *Boynton Beach Club* (em português, *O Clube da Feliz Idade*, 2005) também é uma comédia romântica, porém com diferenças no elenco, direção, locações. É possível que as palavras-chave (keywords) e o overview desses dois filmes sejam bem similares. 

#### <font color='orangered'>Top 3 filmes recomendados para quem assistiu "Sideways - Entre umas e outras"</font>

In [31]:
sistema_recomendacao('Sideways')

Bottle Shock
Boynton Beach Club
Punch-Drunk Love


**Comentário:** 
- O roteiro do primeiro lugar, *Bottle Shock* (*O Julgamento de Paris*, 2008) é muito parecido com o de *Sideways*.
- O terceiro lugar, *Punch-Drunk Love* (*Embriagado de Amor*, 2002) também apresenta muitas similaridades.

#### <font color='orangered'>Top 3 filmes recomendados para quem assistiu "As Aventuras de Paddington"</font>

In [32]:
sistema_recomendacao('Paddington')

The Country Bears
Ted
Dr. Dolittle 2


**Comentário:** 
- Igual a *Paddington*, os 3 filmes recomendados possuem ursos no elenco principal.

## <font color='darkblue'>Conclusão:</font>
- De forma geral, as recomendações foram muito boas para os filmes analisados, com uma ou outra exceção aparente, como as feitas para 1º e 2º lugares de *Um Lugar Chamado Notting Hill*. Mesmo neste caso, é possível que as similaridades estejam associadas com o *overview* e as *keywords* dos filmes recomendados. 

**NOTA:** Este código foi adaptado do curso que estou fazendo pela DSA: *Matemática e Estatística Aplicada para Data Science, Machine Learning e IA*.